In [2]:
# Pt proiect, mai bine ne chinuim cu SVM decat cu k-NN
# Pt proiect, daca vrem sa folosim Kernel, mai ok ar fi cu metoda rbf decat cea liniara

# Pt colocviu, sa ne uitam pe Curs4 -> Forma primala/duala; ar fi ok sa ne uitam pe tot cursul 4 ca la ultimul colocviu a dat exact din el.

In [3]:
# SVM - hard margin -> (nu greseste deloc in separarea datelor)
# SVM - soft margin -> (poate gresi putin, dar e mai ok distanta per total (uitandu-ne la toate datele))

In [4]:
import numpy as np

In [5]:
# preprocesare pt proiect/colocviu:
from sklearn import preprocessing


from sklearn import svm

In [6]:
# functie ok/utila la colocviu/proiect
def normalize_data(train_data, test_data, type=None):
    scaler = None
    if type == 'standard':
        scaler = preprocessing.StandardScaler()


    elif type == 'l1':
        scaler = preprocessing.Normalizer(norm='l1')

    elif type == 'l2':
        scaler = preprocessing.Normalizer(norm='l2')

    if scaler is not None:
        scaler.fit(train_data)
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data) 
        return (scaled_train_data, scaled_test_data)
    else:
        print("No scaling was performed. Raw data is returned.")
        return (train_data, test_data)

    
def accuracy_score(y_true, y_pred):
    return (y_true == y_pred).mean()

In [7]:
###################################################################

In [8]:
# load data
training_data = np.load('data/training_sentences.npy', allow_pickle=True)
training_labels = np.load('data/training_labels.npy')

test_data = np.load('data/test_sentences.npy', allow_pickle=True)
test_labels = np.load('data/test_labels.npy')

In [9]:
training_labels = training_labels.astype(int)

In [10]:
print(training_data)

[list(['Probably', 'not', 'still', 'going', 'over', 'some', 'stuff', 'here'])
 list(['I', 'HAVE', 'A', 'DATE', 'ON', 'SUNDAY', 'WITH', 'WILL'])
 list(['Thanks', '4', 'your', 'continued', 'support', 'Your', 'question', 'this', 'week', 'will', 'enter', 'u', 'in2', 'our', 'draw', '4', 'Â£100', 'cash', 'Name', 'the', 'NEW', 'US', 'President', 'txt', 'ans', 'to', '80082'])
 ...
 list(['OH', 'FUCK', 'JUSWOKE', 'UP', 'IN', 'A', 'BED', 'ON', 'A', 'BOATIN', 'THE', 'DOCKS', 'SLEPT', 'WID', '25', 'YEAR', 'OLD', 'SPINOUT', 'GIV', 'U', 'DA', 'GOSSIP', 'L8R', 'XXX'])
 list(['NOT', 'MUCH', 'NO', 'FIGHTS', 'IT', 'WAS', 'A', 'GOOD', 'NITE'])
 list(['Did', 'he', 'just', 'say', 'somebody', 'is', 'named', 'tampa'])]


In [11]:
class BagOfWords:
    
    def __init__(self):
        self.voc = dict()
    
    def CreateVoc(self, data):
        for document in data:
            for word in document:
                if word in self.voc:
                    continue
                self.voc[word] = len(self.voc)
    
    def GetFeatures(self, data):
        features = np.zeros((len(data), len(self.voc)))
        for i, document in enumerate(data):
            for word in document:
                if word not in self.voc:
                    continue
                features[i][self.voc[word]] += 1 
        return features

In [12]:
bow = BagOfWords()
bow.CreateVoc(training_data)
print(len(bow.voc))

9522


In [20]:
x_train = bow.GetFeatures(training_data)
x_test = bow.GetFeatures(test_data)

print(x_train)
print(x_test)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
scaled_train, scaled_test = normalize_data(x_train, x_test, 'l2')
print(scaled_train)
print(scaled_test)

[[0.35355339 0.35355339 0.35355339 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.19611614 0.19611614 0.        ]
 [0.         0.         0.         ... 0.         0.         0.33333333]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0.  0.  0.5 ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]


In [15]:
# Pt proeict: Cum alegem combinatia (C, kernel)? la fel ca la (k, metoda): Luam mai multe combinatii, afisam cat a dat
# pt fiecare si spunem care din alea ar fi mai buna -> pe aia o luam.
svmClf = svm.SVC(C = 10, kernel = 'linear')
svmClf.fit(scaled_train, training_labels)

SVC(C=10, kernel='linear')

In [16]:
predicted = svmClf.predict(scaled_test)

In [17]:
print(accuracy_score(predicted, test_labels))

0.9891304347826086


In [18]:
print(training_labels)

[0 0 1 ... 0 0 0]
